In [9]:
import numpy as np


pos_features = np.array([2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
neg_features = np.array([3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3])
pos_labels=np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0])
neg_labels=np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1])
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

In [10]:
res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

(20,)

In [11]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

(40,)

In [12]:
resampled_features

array([3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 3, 2, 3, 2, 2,
       3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, 2, 2, 3, 3, 2, 3, 2])

In [14]:
resampled_labels.mean()

0.5